## SQL Coding Questions from Well-known Companies

In [2]:
import pandas as pd
import numpy as np
import sqlite3 as sql

database = "C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db"
connection = sql.connect(database)

query = '''SELECT * FROM airbnb_hosts'''
df = pd.read_sql_query(query, connection)

%load_ext sql
%sql sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db

'Connected: @C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db'

### Airbnb

##### 1. Favorite Host Nationality
For each guest reviewer, find the nationality of the reviewer’s favorite host based on the guest’s highest review score given to a host. <br>
Output the user ID of the guest along with their favorite host’s nationality.
Both the `from_user` and `to_user` columns are user IDs.

tables <br>
airbnb_reviews<br>
airbnb_hosts

In [312]:
%%sql
SELECT DISTINCT(from_user), nationality
FROM airbnb_reviews r LEFT JOIN airbnb_hosts h ON r.to_user=h.host_id
WHERE from_type = 'guest' AND review_score IN (SELECT MAX(review_score) FROM airbnb_reviews)

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


from_user,nationality
6,China
7,China
5,Mali
2,Australia
5,Australia
6,USA
8,Mali
3,Australia
8,USA
7,Luxembourg


##### 2. Accommodates-To-Bed Ratio
Find the average accommodates-to-beds ratio for shared rooms in each city. Sort your results by listing cities with the highest ratios first.

tables <br>
airbnb_search_details

In [330]:
%%sql
SELECT city, AVG(accommodates/beds) avg_crowdness_ratio
FROM airbnb_search_details
WHERE room_type = 'Shared room'
GROUP BY city
ORDER BY 2

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


city,avg_crowdness_ratio
NYC,1.3333333333333333
LA,1.5


##### 3. Cheapest Neighborhoods With Real Beds And Internet
Find neighborhoods where you can sleep on a real bed in a villa with internet while paying the lowest price possible.

tables<br>
airbnb_search_details

In [365]:
%%sql
SELECT neighbourhood
FROM airbnb_search_details
WHERE bed_type = 'Real Bed' AND property_type = 'Villa' AND amenities LIKE '%Internet%' AND price = (SELECT MIN(price) FROM airbnb_search_details where bed_type='Real Bed' AND property_type='Villa' AND amenities LIKE '%Internet%')

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


neighbourhood
Long Beach


##### 4. Ranking Hosts By Beds
Rank each host based on the number of beds they have listed. The host with the most beds should be ranked 1 and the host with the least number of beds should be ranked last.<br> Hosts that have the same number of beds should have the same rank. A host can also own multiple properties. 
Output the host ID, number of beds, and rank from highest rank to lowest.

tables<br>
airbnb_apartments

In [371]:
%%sql
SELECT host_id, SUM(n_beds) number_of_beds, DENSE_RANK() OVER (ORDER BY SUM(n_beds) DESC)
FROM airbnb_apartments
GROUP BY host_id

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


host_id,number_of_beds,DENSE_RANK() OVER (ORDER BY sum(n_beds) DESC)
10,16,1
3,8,2
6,6,3
5,5,4
9,4,5
7,4,5
1,4,5
2,3,6
0,3,6
11,2,7


##### 5. Total Searches For Rooms
Find the total number of searches for each room type (apartments, private, shared) by city.<br>

tables<br>
airbnb_search_details

In [7]:
%%sql 
select city, 
    count(case when room_type = 'Entire home/apt' then id END) as "Entire home/apt",
    count(case when room_type = 'Private room' then id END) as "Private room",
    count(case when room_type = 'Shared room'then id END) as "Shared room"
from airbnb_search_details
group by city

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


city,Entire home/apt,Private room,Shared room
Boston,1,3,0
Chicago,3,1,0
DC,0,1,0
LA,34,25,4
NYC,43,35,3
SF,3,4,0


In [374]:
%%sql 
select city, 
    SUM(IF(room_type = 'Entire home/apt',1,0)) as "Entire home/apt",
    SUM(IF(room_type = 'Private room',1,0)) as "Private room",
    SUM(IF(room_type = 'Shared room',1,0)) as "Shared room"
from airbnb_search_details
group by city

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
(sqlite3.OperationalError) no such function: IF
[SQL: select city, 
    SUM(IF(room_type = 'Entire home/apt',1,0)) as "Entire home/apt",
    SUM(IF(room_type = 'Private room',1,0)) as "Private room",
      SUM(IF(room_type = 'Shared room',1,0)) as "Shared room"
from airbnb_search_details
group by city]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


##### 6. Reviews Bins on Reviews Number (MEDIUM)
To better understand the effect of the review count on the price, categorize the number of reviews into the following groups along with the price.<br>
    0 reviews: NO<br>
    1 to 5 reviews: FEWM<br>
    6 to 15 reviews: SOME<br>
    16 to 40 reviews: MANY<br>
    more than 40 reviews: A LOT<br>

tables
airbnb_search_details

In [8]:
%%sql
select
    price,
    CASE
        WHEN number_of_reviews = 0 THEN 'NO'
        WHEN number_of_reviews BETWEEN 1 AND 5 THEN 'FEW'
        WHEN number_of_reviews BETWEEN 6 AND 15 THEN 'SOME'
        WHEN number_of_reviews BETWEEN 16 AND 40 THEN 'MANY'
        ELSE 'A LOT'
    END AS 'num'
from airbnb_search_details
limit 3

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


price,num
555.68,FEW
366.36,SOME
482.83,A LOT


##### 7. Host Popularity Rental Prices (HARD)
You’re given a table of rental property searches by users. The table consists of search results and outputs host information for searchers. Find the minimum, average, maximum rental prices for each host’s popularity rating. The host’s popularity rating is defined as below:
    0 reviews: New
    1 to 5 reviews: Rising
    6 to 15 reviews: Trending Up
    16 to 40 reviews: Popular
    more than 40 reviews: Hot

Tip: The `id` column in the table refers to the search ID. You'll need to create your own host_id by concating price, room_type, host_since, zipcode, and number_of_reviews.

Output host popularity rating and their minimum, average and maximum rental prices.

* CONCAT does not work in sqlite. so use || like "price||room_type||host_since||zipcode||number_of_reviews"

In [209]:
%%sql
with t1 as (
select 
distinct (price||room_type||host_since||zipcode||number_of_reviews) as host_id,
price,
CASE 
        WHEN number_of_reviews = 0 THEN 'NEW'
        WHEN number_of_reviews BETWEEN 1 AND 5 THEN 'Rising'
        WHEN number_of_reviews BETWEEN 6 AND 15 THEN 'Trending Up'
        WHEN number_of_reviews BETWEEN 16 AND 40 THEN 'Popular'
        ELSE 'Hot' 
        END AS rating_category
from airbnb_host_searches)
select rating_category,min(price),avg(price),max(price) 
from t1
group by rating_category;

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


rating_category,min(price),avg(price),max(price)
Hot,340.12,464.23315789473685,633.51
NEW,313.55,515.9197142857143,741.76
Popular,270.81,472.815,667.83
Rising,355.53,503.8465853658536,717.01
Trending Up,361.09,476.2771794871794,685.65


##### 8. Growth of Airbnb (HARD)
Estimate the growth of Airbnb each year using the number of hosts registered as the growth metric.<br> The rate of growth is calculated by taking ((number of hosts registered in the current year - number of hosts registered in the previous year) / the number of hosts registered in the previous year) * 100.
<br>Output the year, number of hosts in the current year, number of hosts in the previous year, and the rate of growth. Round the rate of growth to the nearest percent and order the result in the ascending order based on the year.
Assume that the dataset consists only of unique hosts, meaning there are no duplicate hosts listed.

* sqlite has no YEAR function. so use strftime('%Y', '2035-12-01') AS "Year"

In [276]:
%%sql
select year , curr , prev , Round(((curr -prev)/prev) * 100 ,0) as growth
from
(select strftime('%Y', host_since) as year, count(id) as curr, lag(count(id) , 1)over (order by strftime('%Y', host_since)) as prev 
from
airbnb_search_details
group by 1
order by 1)

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


year,curr,prev,growth
2009,2,None,None
2010,4,2,100.0
2011,9,4,100.0
2012,10,9,0.0
2013,30,10,200.0
2014,33,30,0.0
2015,33,33,0.0
2016,28,33,0.0
2017,11,28,0.0


##### 9. City With Most Amenities
You're given a dataset of searches for properties on Airbnb. For simplicity, let's say that each search result (i.e., each row) represents a unique host. Find the city with the most amenities across all their host's properties. Output the name of the city.

In [281]:
%%sql
SELECT city
FROM (SELECT city,
      LENGTH(amenities) - LENGTH(replace(amenities,',','')) as length
      from airbnb_search_details)
WHERE MAX(length)

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
(sqlite3.OperationalError) misuse of aggregate function MAX()
[SQL: SELECT city
FROM (SELECT city,
      LENGTH(amenities) - LENGTH(replace(amenities,',','')) as length
      from airbnb_search_details
      GROUP BY city)
WHERE MAX(length)]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


##### 10. Host Response Rates With Cleaning Fees
Find the average host response rate with a cleaning fee for each zipcode. Present the results as a percentage along with the zip code value.<br> 
Convert the column 'host_response_rate' from TEXT to NUMERIC using type casts and string processing (take missing values as NULL). 
<br>Order the result in ascending order based on the average host response rater after cleaning.

In [295]:
%%sql
SELECT zipcode, AVG(CAST(host_response_rate AS signed)) 
FROM airbnb_search_details
WHERE cleaning_fee = 'TRUE' AND host_response_rate IS NOT NULL
GROUP BY zipcode
ORDER BY 2

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


zipcode,AVG(CAST(host_response_rate AS signed))
91324,0.0
90028,25.0
10035,67.0
90703,70.0
10039,76.0
90265,78.5
10065,80.0
11106,80.0
90057,80.0
10027,88.42857142857143


##### 11. Find the number of unique properties
Find the number of unique properties in the dataset.

In [300]:
%%sql
SELECT DISTINCT(ds||id_user||ds_checkin||ds_checkout||n_searches||n_nights||n_guests_min||n_guests_max||origin_country||filter_price_min||filter_price_max||filter_room_types||filter_neighborhoods
      FROM airbnb_searches

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
(sqlite3.OperationalError) near "FROM": syntax error
[SQL: SELECT DISTINCT(ds||id_user||ds_checkin||ds_checkout||n_searches||n_nights||n_guests_min||n_guests_max||origin_country||filter_price_min||filter_price_max||filter_room_types||filter_neighborhoods
      FROM airbnb_searches]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [299]:
%%sql
SELECT COUNT(*)
FROM (SELECT DISTINCT(ds||id_user||ds_checkin||ds_checkout||n_searches||n_nights||n_guests_min||n_guests_max||origin_country||filter_price_min||filter_price_max||filter_room_types||filter_neighborhoods
      FROM airbnb_searches)

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
(sqlite3.OperationalError) near "FROM": syntax error
[SQL: SELECT COUNT(*)
FROM (SELECT DISTINCT(ds||id_user||ds_checkin||ds_checkout||n_searches||n_nights||n_guests_min||n_guests_max||origin_country||filter_price_min||filter_price_max||filter_room_types||filter_neighborhoods
      FROM airbnb_searches)]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


### Uber

##### 1. Highest Total Miles
You’re given a table of Uber rides that contains the mileage and the purpose for the business expense.  You’re asked to find business purposes that generate the most miles driven for passengers that use Uber for their business transportation. Find the top 3 business purpose categories by total mileage.

In [9]:
%%sql
SELECT purpose, SUM(miles) 'total miles'
FROM my_uber_drives
WHERE category = 'Business'
GROUP BY purpose
ORDER BY 2 desc
limit 3

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


purpose,total miles
Meeting,283.9
Customer Visit,241.09999999999997
Meal/Entertain,77.99999999999997


### Apple

##### 1. Department Salaries
Find the number of male and female employees per department and also their corresponding total salaries.
Output department names along with the corresponding number of female employees, the total salary of female employees, the number of male employees, and the total salary of male employees.

In [10]:
%%sql
SELECT 
    department,
    SUM(CASE WHEN sex = 'F' THEN 1 ELSE 0 END) FEMALE,
    SUM(CASE WHEN sex = 'F' THEN salary ELSE 0 END) FEMALE_SALARY,
    SUM(CASE WHEN sex = 'M' THEN 1 ELSE 0 END) MALE,
    SUM(CASE WHEN sex = 'M' THEN salary ELSE 0 END) MALE_SALARY
FROM apple_employee
GROUP BY 1

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


department,FEMALE,FEMALE_SALARY,MALE,MALE_SALARY
Audit,2,1700,2,2100
Management,2,350000,2,350000
Sales,9,11600,13,17800


##### 2. Customers Without Orders
Find customers who have never made an order.
Output the first name of the customer.

In [11]:
%%sql
SELECT first_name
FROM apple_customers c LEFT JOIN apple_orders o ON c.id = o.cust_id
WHERE total_order_cost IS NULL

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


first_name
John
Emma
Liam
Mark
Jack
Mona
Lili
Justin
Frank


In [12]:
%%sql
select first_name 
from apple_customers 
where id not in (select distinct cust_id from apple_orders)

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


first_name
John
Emma
Liam
Mark
Jack
Mona
Lili
Justin
Frank


##### 3. Duplicate Orders
Find customers who appear in the orders table more than three times.

In [30]:
%%sql
SELECT cust_id 
FROM apple_orders
GROUP BY 1
HAVING COUNT(cust_id) >3

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


cust_id
3
7
15


### Twitter

##### 1. Highest Salary In Department
Find the employee with the highest salary per department.
Output the department name, employee's first name along with the corresponding salary.

In [41]:
%%sql
SELECT department, first_name, salary
FROM twitter_employee
WHERE salary IN (SELECT MAX(salary) FROM twitter_employee GROUP BY department)


 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


department,first_name,salary
Management,Richerd,250000
Sales,Mick,2200
Audit,Shandler,1100


### Amazon

##### 1. Finding User Purchases
Write a query that'll identify returning active users. A returning active user is a user that has made a second purchase within 7 days of any other of their purchases. Output a list of user_ids of these returning active users.

In [62]:
%%sql
SELECT distinct(a.user_id)
FROM amazon_transactions a, amazon_transactions b
WHERE a.user_id = b.user_id AND ABS(DATEDIFF(a.created_at,b.created_at)) <=7 AND a.id != b.id

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
(sqlite3.OperationalError) no such function: DATEDIFF
[SQL: SELECT distinct(a.user_id)
FROM amazon_transactions a, amazon_transactions b
WHERE a.user_id = b.user_id AND ABS(DATEDIFF(a.created_at,b.created_at)) <=7 AND a.id != b.id]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


##### 2. Highest Cost Orders
Find the customer with the highest daily total order cost between 2019-02-01 to 2019-05-01. If customer had more than one order on a certain day, sum the order costs on daily basis. Output their first name, total cost of their items, and the date.
 
 For simplicity, you can assume that every first name in the dataset is unique.

In [70]:
%%sql
SELECT first_name, SUM(total_order_cost), order_date
FROM amazon_customers c LEFT JOIN amazon_orders o ON c.id=o.cust_id
WHERE order_date BETWEEN '2019-02-01' AND '2019-05-01'
GROUP BY first_name, order_date
ORDER BY 2 desc
LIMIT 1


 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


first_name,SUM(total_order_cost),order_date
Jill,275,2019-04-19


##### 3. Order Details
Find order details made by Jill and Eva.
Consider the Jill and Eva as first names of customers. </p>
Output the order date, details and cost along with the first name.
Order records based on the customer id in ascending order.

In [75]:
%%sql
SELECT order_date, order_details, total_order_cost, first_name
FROM amazon_orders o LEFT JOIN amazon_customers c ON o.cust_id=c.id
WHERE first_name IN ('Jill', 'Eva')
ORDER BY cust_id

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


order_date,order_details,total_order_cost,first_name
2019-02-01,Coat,25,Jill
2019-03-10,Shoes,80,Jill
2019-04-19,Suit,150,Jill
2019-04-01,Suit,50,Jill
2019-04-02,Skirt,30,Jill
2019-04-03,Dresses,50,Jill
2019-04-04,Coat,25,Jill
2019-04-19,Coat,125,Jill
2019-01-11,Shirts,60,Eva
2019-03-11,Slipper,20,Eva


##### 4. Customer Orders and Details
Find the number of orders, the number of customers, and the total cost of orders for each city. Only include cities that have made at least 5 orders and count all customers in each city even if they did not place an order.

Output each calculation along with the corresponding city name.

In [110]:
%%sql
SELECT city, COUNT(o.id) orders, COUNT(distinct c.id) customers, SUM(total_order_cost) total_cost
FROM amazon_customers c LEFT JOIN amazon_orders o ON c.id = o.cust_id
GROUP BY city
HAVING orders > 4


 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


city,orders,customers,total_cost
Austin,8,1,535
Miami,8,5,620


##### 5. Favorite Customer
Find "favorite" customers based on the order count and the total cost of orders.
A customer is considered as a favorite if he or she has placed more than 3 orders and with the total cost of orders more than $100.

Output the customer's first name, city, number of orders, and total cost of orders.

In [111]:
%%sql
SELECT first_name, city, COUNT(o.id) order_count, SUM(total_order_cost)
FROM amazon_orders o LEFT JOIN amazon_customers c ON o.cust_id = c.id
GROUP BY first_name
HAVING order_count > 3 AND SUM(total_order_cost) > 100


 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


first_name,city,order_count,SUM(total_order_cost)
Farida,San Francisco,4,260
Jill,Austin,8,535
Mia,Miami,7,540


##### 6. Lowest Priced Orders
Find the lowest order cost of each customer.
Output the customer id along with the first name and the lowest order price.

In [114]:
%%sql
SELECT c.id, first_name, MIN(total_order_cost)
FROM amazon_orders o LEFT JOIN amazon_customers c ON o.cust_id=c.id
GROUP BY c.id

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


id,first_name,MIN(total_order_cost)
3,Farida,30
4,William,60
5,Henry,80
7,Jill,25
12,Eva,20
15,Mia,20


##### 7. Second Highest Salary
Find the second highest salary of employees.

In [122]:
%%sql
SELECT salary 
FROM amazon_employee
ORDER BY salary desc
LIMIT 1 OFFSET 1

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


salary
200000


In [123]:
%%sql
SELECT salary 
FROM (select salary, rank() OVER (ORDER BY salary DESC) AS r FROM amazon_employee) as temp
where temp.r = 2

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


salary
200000


### City of Los Angeles

##### 1. Owners With 3 Grades
Find the owners who have at least one facility with all 3 grades.

In [158]:
%%sql
SELECT owner_name
FROM la_restaurant_health_inspections
GROUP BY owner_name, facility_name
HAVING COUNT(DISTINCT grade) = 3

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


owner_name
GOODWIN TYRONE
ROUND 3 INC.


##### 2. Classifying Owner Names
Classify owner names as LLC, INC and OTHER using the below classification.

    If owner name contains 'LLC' or 'LL', then classify them as 'LLC'.
    If owner name contains 'INC', then classify them as 'INC'.
    Classify others as 'OTHER'.

In [164]:
%%sql
SELECT DISTINCT owner_name, 
    CASE WHEN owner_name LIKE '%LLC%' OR owner_name LIKE '%LL%' THEN 'LLC'
    WHEN owner_name LIKE '%INC%' THEN 'INC'
    ELSE 'OTHER'
    END AS classification
FROM los_angeles_restaurant_health_inspections

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


owner_name,classification
"BAZAN, ASCENCION",OTHER
MARISOL FEREGRINO,OTHER
SANDRA'S TAMALES INC.,INC
CAFE GRATITUDE LARCHMONT LLC,LLC
"THE WAFFLE, LLC",LLC
MARIN & MARTINEZ GROUP CORP.,OTHER
UCLAWP LLC,LLC
EMME LAN P. DIEU,OTHER
YOUNG YOON,OTHER
LENTO MARKET LLC,LLC


##### 3. Find the first and last times the maximum score was awarded
Find the first and last times the maximum score was awarded

In [167]:
%%sql
select min(activity_date), max(activity_date)
from los_angeles_restaurant_health_inspections
where score = (select max(score) from los_angeles_restaurant_health_inspections )

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


min(activity_date),max(activity_date)
2015-09-11,2018-03-16


##### 4. Rules To Determine Grades
Find the rules used to determine each grade. Show the rule in a separate column in the format of 'Score > X AND Score <= Y => Grade = A' where X and Y are the lower and upper bounds for a grade. Output the corresponding grade and its highest and lowest scores along with the rule. Order the result based on the grade in ascending order.

In [174]:
%%sql
select DISTINCT grade, MIN(score), MAX(score),
CONCAT('Score > ', MIN(score) - 1,' AND Score <= ', MAX(score), ' => Grade = ', grade) as rule
from los_angeles_restaurant_health_inspections
GROUP BY grade
ORDER BY 1

 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
(sqlite3.OperationalError) no such function: CONCAT
[SQL: select DISTINCT grade, MIN(score), MAX(score),
CONCAT('Score > ', MIN(score) - 1,' AND Score <= ', MAX(score), ' => Grade = ', grade) as rule
from los_angeles_restaurant_health_inspections
GROUP BY grade
ORDER BY 1]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


##### 5. Find the total number of inspections with low risk in 2017
Find the total number of inspections with low risk in 2017.

In [179]:
%%sql 
SELECT COUNT(pe_description)
FROM los_angeles_restaurant_health_inspections
WHERE activity_date LIKE '%2017%' AND pe_description LIKE '%LOW RISK%'


 * sqlite:///C:/Users/persi/Desktop/서웅/DataScience/Portfolio/Portfolio/MySQL/data/conn.db
Done.


COUNT(pe_description)
18
